# Development notes for pyxelate

## Display ratio calculation

Round to quarters for ratios below 3, to halves for ratios below 5, and to integer numbers for all greater ratios.

In [1]:
import math
from fractions import Fraction

uproundable = Fraction(17, 10)
exact = Fraction(15, 10)

uprounded_ratio = Fraction(math.ceil(uproundable * 4), 4)
exact_ratio = Fraction(math.ceil(exact * 4), 4)

print('Uprounded %s: %s' % (uproundable, uprounded_ratio))
print('Exact %s: %s' % (exact, exact_ratio))

Uprounded 17/10: 7/4
Exact 3/2: 3/2


In [3]:
def display_ratio(image_size, canvas_size):
    """Display ratio calculated per dimension"""
    if image_size < canvas_size:
        return 1
    #
    raw_ratio = Fraction(image_size, canvas_size)
    if raw_ratio <= 3:
        return Fraction(math.ceil(raw_ratio * 4), 4)
    #
    if raw_ratio <= 5:
        return Fraction(math.ceil(raw_ratio * 2), 2)
    #
    return math.ceil(raw_ratio)

In [4]:
print(display_ratio(17, 10))
print(display_ratio(17, 5))
print(display_ratio(17, 3))

7/4
7/2
6


## pixelate-video considerations

### Add a ttk.progressbar in a modal window for long-running tasks

<https://tkdocs.com/shipman/ttk-Progressbar.html> with `mode='determinate'` if possible

Use cases:
- Splitting a video file into frames (after selecting a video file)
- Applying pixelations to the selected frames (apply and continue, save as…)
- Joining the frames to generate the result video (save as…)
- Adding the audio stream to the result video (save as…)

### Generic workflow

- Select a video file
- Create a temporary directory (cleanup an existing one before)
- Split the video into frames -> `{tempdir}/original` directory (see <https://github.com/Wikinaut/utils/wiki#How_to_extract_every_nth_frame>))
- Select the first frame for pixelation using a scale (<https://tkdocs.com/shipman/scale.html>, ending the last but one frame) and select the pixelation parameters like in pixelate_image
- Select the last frame for pixelation using a scale (starting at the frame directly following the first frame) and select pixelation parameters _except tilesize and shape_
- When clicking 'Apply and continue':
  - Apply the pixelations from start frame to end frame
  - Replace the matching originals with these frames
  - Start over at selecting the first frame
- When clicking 'Save as…':
  - Apply the pixelations from start frame to end frame
  - Join all frames into a new video, re-using all applicable information from the original video
  - Add the original audio stream(s) of the original video, if any
  - Cleanup the temporary directory

### "Touched" flag

- Set to `False` after loading a video
- Set to `True` after first change on the "Select first frame" panel
- Set to `False` after saving